<a href="https://colab.research.google.com/github/JY522IC/PointCloudProject/blob/main/Over_Sampling_Pointnet_classification_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the required libraries

In [ ]:
# install the required libraries
!pip install open3d
!pip install provider

# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-cluster
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper functions for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_mesh(pos, face):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    # ax = fig.gca(projection='3d')
    ax.axes.xaxis.set_ticklabels([])
    ax.axes.yaxis.set_ticklabels([])
    ax.axes.zaxis.set_ticklabels([])
    ax.plot_trisurf(pos[:, 0], pos[:, 1], pos[:, 2], triangles=data.face.t(), antialiased=False)
    plt.show()


def visualize_points(pos, edge_index=None, index=None):
    fig = plt.figure(figsize=(4, 4))
    if edge_index is not None:
        for (src, dst) in edge_index.t().tolist():
             src = pos[src].tolist()
             dst = pos[dst].tolist()
             plt.plot([src[0], dst[0]], [src[1], dst[1]], linewidth=1, color='black')
    if index is None:
        plt.scatter(pos[:, 0], pos[:, 1], s=50, zorder=1000)
    else:
       mask = torch.zeros(pos.size(0), dtype=torch.bool)
       mask[index] = True
       plt.scatter(pos[~mask, 0], pos[~mask, 1], s=50, color='lightgray', zorder=1000)
       plt.scatter(pos[mask, 0], pos[mask, 1], s=50, zorder=1000)
    plt.axis('off')
    plt.show()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.0.1+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch_cluster-1.6.1.tar.gz (53 kB)
  Preparing metadata (setup.py) ... done


# Import the required libraries

In [ ]:
import torch
import os
import pandas as pd
from torch_geometric.data import InMemoryDataset, Data, download_url, extract_zip
from torch_geometric.utils.convert import to_networkx
import networkx as nx
from tqdm import tqdm
import open3d as o3d
import numpy as np

KeyboardInterrupt: ignored

# The function to load the data and return positions in the dataset

In [ ]:
# input the file path and return the data of all labels
def loadData(path):
    # Load the pcd file
    pcd = o3d.io.read_point_cloud(path)

    # Control the number of points in the point cloud file
    downpcd_farthest = pcd.farthest_point_down_sample(5000)

    point_xyz = np.asarray(downpcd_farthest.points)

    return point_xyz

# Do the point cloud normilization to make the point cloud to be around the center point

In [ ]:
def pc_normalize(pc):
    """
    Point data normilization
    :param pc: the pc to normilized
    :return: the point data after normilization
    """
    # the average value
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc ** 2, axis=1)))
    # rescale
    pc = pc / m
    return pc

# Define the dataset using Pytorch Geometry

In [ ]:
class Operating_room_classification(InMemoryDataset):

    # Base url to download the files
    # url = 'http://nrvis.com/download/data/labeled/FRANKENSTEIN.zip'

    def __init__(self, root, transform=None, pre_transform=None):
        super(Operating_room_classification, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        # List of the raw files
        return []

    @property
    def processed_file_names(self):
        # return list of files should be in processed dir, if found - skip processing
        return ['data.pt']

    # def download(self):
    #     # Download the file specified in self.url and store
    #     # it in self.raw_dir
    #     path = download_url(self.url, self.raw_dir)
    #     extract_zip(path, self.raw_dir)
    #     # The zip file is removed
    #     os.unlink(path)


    def process(self):

      if State == 1:
        df = train_index
      else:
        df = test_index

      # In the loop we extract the nodes' embeddings, edges connectivity for
      # and label for a graph, process the information and put it in a Data
      # object, then we add the object to a list
      data_list = []
      root_new = '/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/Down20000 4'

      for index, row in df.iterrows():

        # load the data
        point_pos = loadData(os.path.join(root_new, row[1]))

        # Normilize the data
        new_points = pc_normalize(point_pos)

        # Calculate the normal information
        device = o3d.core.Device("CPU:0")
        dtype = o3d.core.float64
        pcd = o3d.t.geometry.PointCloud(device)
        pcd.point.positions = o3d.core.Tensor(new_points, dtype, device)
        pcd = pcd.to_legacy()
        radius = 0.01  # Searching Radius
        max_nn = 30  # Number of neighboring points to estimate normal vector
        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))

        # Convert the position and label information into the Pytorch format
        Pos = torch.tensor(new_points,dtype=torch.float32)
        Y = torch.tensor(row[2],dtype=torch.long)
        Normals = torch.tensor(np.asarray(pcd.normals),dtype=torch.float32)

        # Save the data
        graph = Data(pos = Pos, y = Y, normal = Normals)
        data_list.append(graph)

        print(index)

      # Apply the functions specified in pre_filter and pre_transform
      if self.pre_filter is not None:
          data_list = [data for data in data_list if self.pre_filter(data)]

      if self.pre_transform is not None:
          data_list = [self.pre_transform(data) for data in data_list]

      # Store the processed data
      data, slices = self.collate(data_list)
      torch.save((data, slices), self.processed_paths[0])

# Create the dataset

In [ ]:
import torch_geometric.transforms as T

# load data
df = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/Two Class Down20000 4 with label.csv')
df = df.sample(frac=1).reset_index(drop=True)
train_index=df.sample(frac=0.7)
test_index=df[~df.index.isin(train_index.index)]

# Create the train and test dataset based on the direction
State = 1
trainDataset = Operating_room_classification(root='/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/Down20000 4/SittingPreparingOperating/Train')
State = 2
testDataset = Operating_room_classification(root='/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/Down20000 4/SittingPreparingOperating/Test')

# Rescale the point to its center position

In [ ]:
import open3d.core as o3c

# Load the point cloud data
data_1 = o3d.io.read_point_cloud('/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/Down20000 4/1380.ply')

# Visualize the point cloud
o3d.visualization.draw_plotly([data_1])

# Normalize the point cloud data
new_points = pc_normalize(np.asarray(data_1.points))

# print(new_points)

# Define the open3d format and visualize the data
device = o3d.core.Device("CPU:0")
dtype = o3d.core.float64

# Create an empty point cloud
# Use pcd.point to access the points' attributes,
pcd = o3d.t.geometry.PointCloud(device)
# pcd = o3d.t.geometry.PointCloud()

# Default attribute: "positions".
# This attribute is created by default and is required by all point clouds.
# The shape must be (N, 3). The device of "positions" determines the device
# of the point cloud.
pcd.point.positions = o3d.core.Tensor(new_points, dtype, device)
pcd.point.normals = o3d.core.Tensor(np.asarray(data_1.normals), dtype, device)
pcd.point.colors = o3d.core.Tensor(np.asarray(data_1.colors), dtype, device)
pcd = pcd.to_legacy()
o3d.visualization.draw_plotly([pcd])

## Define the Neural Network

Pointnet Layer

In [ ]:
# Pointnet layer
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import MessagePassing


class PointNetLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        # Message passing with "max" aggregation.
        super().__init__(aggr='max')

        # Initialization of the MLP:
        # Here, the number of input features correspond to the hidden node
        # dimensionality plus point dimensionality (=3).
        self.mlp = Sequential(Linear(in_channels+3, out_channels),
                              ReLU(),
                              Linear(out_channels, out_channels))

    def forward(self, h, pos, edge_index):
        # Start propagating messages.
        return self.propagate(edge_index, h=h, pos=pos)

    def message(self, h_j, pos_j, pos_i):
        # h_j defines the features of neighboring nodes as shape [num_edges, in_channels]
        # pos_j defines the position of neighboring nodes as shape [num_edges, 3]
        # pos_i defines the position of central nodes as shape [num_edges, 3]

        # Compute spatial relation.
        input = pos_j - pos_i

        input = input.to(torch.float32)

        if h_j is not None:
            # In the first layer, we may not have any hidden node features,
            # so we only combine them in case they are present.
            input = torch.cat([h_j, input], dim=-1)

        return self.mlp(input)  # Apply our final MLP.

Main network structure

In [ ]:
from torch_geometric.nn.conv import edge_conv
import torch
import torch.nn.functional as F
from torch_cluster import knn_graph
from torch_geometric.nn import global_max_pool

class PointNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        torch.manual_seed(12345)
        self.conv1 = PointNetLayer(3, 32)
        self.conv2 = PointNetLayer(32, 32)
        self.classifier = Linear(32, trainDataset.num_classes)

    def forward(self, pos, batch):
        # Compute the kNN graph:
        # Here, we need to pass the batch vector to the function call in order
        # to prevent creating edges between points of different examples.
        # We also add `loop=True` which will add self-loops to the graph in
        # order to preserve central point information.
        edge_index = knn_graph(pos, k=16, batch=batch, loop=True)
        # print(edge_index)
        # print(edge_index.shape)
        # print(edge_index.type)

        # Start bipartite message passing.
        h = self.conv1(h=pos, pos=pos, edge_index=edge_index)

        h = h.relu()

        h = self.conv2(h=h, pos=pos, edge_index=edge_index)

        h = h.relu()

        # Global Pooling.
        h = global_max_pool(h, batch)  # [num_examples, hidden_channels]

        # Classifier.
        return self.classifier(h)

# Show the network structure
model = PointNet()
print(model)

Train and test

In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

from torch_geometric.loader import DataLoader
import os
import sys
import torch
import numpy as np

import datetime
import logging
import provider
import importlib
import shutil
import argparse

from pathlib import Path
from tqdm import tqdm
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

from sklearn.utils.class_weight import compute_sample_weight
from torch_geometric.loader import DataLoader
from torch.utils.data import WeightedRandomSampler

# train_dataset = GeometricShapes(root='data/GeometricShapes', train=True,
#                                 transform=SamplePoints(128))
# test_dataset = GeometricShapes(root='data/GeometricShapes', train=False,
#                                transform=SamplePoints(128))

# Over Sampling the data
y_label = np.asarray(trainDataset.data.y)
y_weight = compute_sample_weight(class_weight = 'balanced', y = y_label)
Sampler = WeightedRandomSampler(y_weight, len(y_label), replacement=True)

# Load the data
trainDataLoader = DataLoader(trainDataset, batch_size=20, sampler = Sampler)
testDataLoader = DataLoader(testDataset, batch_size=10)

# Load the Pointnet
model = PointNet()
# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# Define loss criterion.
criterion = torch.nn.CrossEntropyLoss()

# Define the log
def log_string(str):
  logger.info(str)
  print(str)


# log_dir = '/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/LogsCoordinateCalibPointnetPlusGnn/classification/2023-05-15_15-47'
log_dir = None
model_name = 'PointNetPlusGNNWithoutCoordinateCalibration'

'''HYPER PARAMETER'''
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

'''CREATE DIR'''
timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
exp_dir = Path('/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/ResultSittingPreparingOperating/LogsPointnetAndGNN')
exp_dir.mkdir(exist_ok=True)
exp_dir = exp_dir.joinpath('classification')
exp_dir.mkdir(exist_ok=True)

if log_dir is None:
    exp_dir = exp_dir.joinpath(timestr)
else:
    exp_dir = exp_dir.joinpath(log_dir)

exp_dir.mkdir(exist_ok=True)
checkpoints_dir = exp_dir.joinpath('checkpoints/')
checkpoints_dir.mkdir(exist_ok=True)
log_dir = exp_dir.joinpath('logs/')
log_dir.mkdir(exist_ok=True)

'''LOG'''
logger = logging.getLogger("Model")
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler('%s/%s.log' % (log_dir, model_name))
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
log_string('PARAMETER ...')

try:
    checkpoint = torch.load(str(exp_dir) + '/checkpoints/best_model.pth')
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    log_string('Use pretrain model')
except:
    log_string('No existing model, starting training from scratch...')
    start_epoch = 0


# Define the train function
def train(model, optimizer, loader):

  # Train model
  model.train()

  total_loss = 0
  # itar = 0

  for data in tqdm(loader,desc='Processing'):
    # itar+= 1
    # print(itar)

    # Clear gradients.
    optimizer.zero_grad()

    # data.pos = data.pos.to(torch.float32)
    # print(data.pos.shape)
    # print(data.batch.shape)

    # Forward pass.
    logits = model(data.pos, data.batch)

    # Loss computation.
    loss = criterion(logits, data.y)

    # Backward pass.
    loss.backward()

    # Update model parameters.
    optimizer.step()
    total_loss += loss.item() * data.num_graphs
    # print(total_loss)

  return total_loss / len(train_loader.dataset)

# Define the test function
@torch.no_grad()
def test(model, loader):

  # Define the eval
  model.eval()

  total_correct = 0
  for data in loader:
    data.pos = data.pos.to(torch.float32)
    logits = model(data.pos, data.batch)
    pred = logits.argmax(dim=-1)
    total_correct += int((pred == data.y).sum())

  return total_correct / len(loader.dataset)

best_instance_acc = 0.0

# Define the main part
'''TRANING'''
logger.info('Start training...')
for epoch in range(1, 20):
    loss = train(model, optimizer, train_loader)
    test_acc = test(model, test_loader)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Test Accuracy: {test_acc:.4f}')

    if (test_acc >= best_instance_acc):
      best_instance_acc = test_acc
      best_epoch = epoch + 1

    # if (class_acc >= best_class_acc):
    #     best_class_acc = class_acc
    log_string('Test Instance Accuracy: %f' % (test_acc))
    log_string('Best Instance Accuracy: %f' % (best_instance_acc))

    if (test_acc >= best_instance_acc):
      logger.info('Save model...')
      savepath = str(checkpoints_dir) + '/best_model.pth'
      log_string('Saving at %s' % savepath)
      state = {
          'epoch': best_epoch,
          'instance_acc': test_acc,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
      }
      torch.save(state, savepath)

logger.info('End of training...')

# Save the model
# PATH = '/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/CoordinateCalibratePointnetPlusModel0504.pt'
# torch.save(model, PATH)

In [ ]:
# Test the model saved
model_new = torch.load('/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/CoordinateCalibratePointnetPlusModel0504.pt')
point_pos = loadData('/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/Down20000 4/77250.ply')
new_points = pc_normalize(point_pos)
device = o3d.core.Device("CPU:0")
dtype = o3d.core.float64

# # Create an empty point cloud
# # Use pcd.point to access the points' attributes
pcd = o3d.t.geometry.PointCloud(device)
# pcd = o3d.t.geometry.PointCloud()

# Default attribute: "positions".
# This attribute is created by default and is required by all point clouds.
# The shape must be (N, 3). The device of "positions" determines the device
# of the point cloud.
pcd.point.positions = o3d.core.Tensor(new_points, dtype, device)
pcd = pcd.to_legacy()
o3d.visualization.draw_plotly([pcd])

test_points = torch.tensor(new_points,dtype=torch.float32)

logits = model_new(test_points,None)
pred = np.asarray(logits.argmax(dim=-1))
print(pred)

In [ ]:
logger = logging.getLogger('test')
logger.setLevel(level=logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s')

file_handler = logging.FileHandler('/content/drive/MyDrive/Colab Notebooks/Pointnet 0428/LogsCoordinateCalibPointnetPlusGnn/test2.log')
file_handler.setLevel(level=logging.INFO)
file_handler.setFormatter(formatter)

stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.DEBUG)
stream_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(stream_handler)